<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [ ]:
"""
network.py
~~~~~~~~~~
A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))






In [1]:
import numpy as np #se importa numpy como np
import random

In [2]:
class RedNeuronal:
    # num_entrada: dimension de los vectores de entrada, en iris: 4
    # tam_capas: lista de cantidad de neuronas por cada capa interna, en iris: [32, 16]
    # num_salida: dimension de los vectores de salida, en iris: 3
    def __init__(self, num_entrada, tam_capas, num_salida):
        self.num_entrada = num_entrada
        self.tam_capas = tam_capas
        self.num_salida = num_salida
        self.capas = []
        # iniciar todas las capas inicial - ocultas - final
        self.capas.append(CapaNeuronal(num_entrada, "sigmoid"))
        for i in range(len(tam_capas)):
            self.capas.append(CapaNeuronal(tam_capas[i], "sigmoid", self.capas[i].tam_capa))
        self.capas.append(CapaNeuronal(num_salida, "softmax", self.capas[len(self.capas)-1].tam_capa))
        self.cant_capas = len(self.capas)
        
    def crossentropy(self, prediccion, esperado, epsilon=1e-12): #El epsilon evita log(0)
        prediccion = np.clip(prediccion, epsilon, 1. - epsilon)
        N = prediccion.shape[0]
        ce = -np.sum(np.sum(esperado*np.log(prediccion+1e-9)))/N
        return ce
    
    def crossentropy_derivative(self, prediccion, esperado):
        return (esperado-1/prediccion+(1+esperado)*(1/(1-prediccion)))  
    
    def cost(self, prediccion, esperado):
        return np.dot(prediccion-esperado, prediccion-esperado)
    
    def cost_derivative(self, prediccion, esperado):
        return prediccion-esperado
    
    def train(self, entrada, salida, ciclos, tasa):
        # repetir para todas 
        last = self.cant_capas-1 # indice capa de salida
        result = zip(entrada,salida)
        resultList = list(result)
        for ciclo in range(ciclos):
            # Comienza 1 epoch
            random.shuffle(resultList)
            errores = []
            for input_v, output in resultList:
                
                self.capas[0].set_capa(input_v, input_v) #Poblar capa inicial
                # Feed forward 
                for j in range(1, len(self.capas)):
                    self.capas[j].feed_forward(self.capas[j-1])
                
                # evaluar error final (delta función costo)
                error_epoch = self.crossentropy(self.capas[last].get_activaciones(), output)
                errores.append(error_epoch)
                #print("Error = "+str(error_epoch))
                ce_derivate = self.crossentropy_derivative(self.capas[last].get_activaciones(), np.transpose(output))
                delta = ce_derivate * self.capas[last].squash_derivative(self.capas[last].get_vector_z())
                print(ce_derivate)
                #Backward pass capa final
                self.capas[last].actualizar_bias(delta, tasa)
                self.capas[last].actualizar_pesos(np.outer(self.capas[last-1].get_activaciones(), delta), tasa)
                #print(self.capas[last].get_pesos())
                #Backward pass general
                for l in range(2, self.cant_capas):
                    z = self.capas[-l].get_vector_z()
                    act_prime = self.capas[-l].squash_derivative(z)
                    delta = np.dot(self.capas[-l+1].get_pesos(), delta.transpose()) * act_prime
                    self.capas[-l].actualizar_bias(delta, tasa)
                    self.capas[-l].actualizar_pesos(np.outer(self.capas[-l-1].get_activaciones(),delta), tasa)
                    
            print("Error del epoch",ciclo,":", str(np.round(np.mean(errores),3)))
            
            #Error de capa final, funcion costo
            



In [3]:
class CapaNeuronal:
    def __init__(self, tam_capa, tipo, tam_anterior = 0):
        self.tam_capa = tam_capa
        self.tam_anterior = tam_anterior
        self.bias = 1
        self.neuronas = [Neurona(tam_anterior) for i in range(tam_capa)] # lista de neuronas de la capa
        if tipo == "sigmoid":
            self.squash = self.sigmoid
            self.squash_derivative = self.sigmoid_derivative
        elif tipo == "softmax":
            self.squash = self.softmax
            self.squash_derivative = self.softmax_derivative
       
    def sigmoid(self, x):
        return np.array([1 / (1 + np.exp(-i)) for i in x])
    
    def sigmoid_derivative(self, x):
        return np.multiply(self.sigmoid(x),(1-self.sigmoid(x)))
    
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)
    
    def softmax_derivative(self, x):
        res = []
        for i in x:
            res.append(np.exp(i)*(np.sum(np.exp(x), axis=0)-np.exp(i))/(np.sum(np.exp(x), axis=0)**2))
        #print(x)
        return np.array(res)
    
    def get_activaciones(self):
        return np.array([neurona.activacion for neurona in self.neuronas])
    
    def get_vector_z(self):
        return [neurona.z for neurona in self.neuronas]
    
    
    def feed_forward(self, capa_anterior):
        pre_squash = []
        activaciones = capa_anterior.get_activaciones()
        vector_z = capa_anterior.get_vector_z()
        for neurona in self.neuronas:
            pesos = neurona.pesos
            pre_squash.append(np.dot(pesos, activaciones) + neurona.bias)
        self.set_capa(self.squash(pre_squash), pre_squash)
        
   
    def get_pesos(self):
        return np.transpose(np.array([n.pesos for n in self.neuronas]))

    def actualizar_pesos(self, matriz, rate):
        pesos = np.transpose(np.array([n.pesos for n in self.neuronas]))
        pesos -= rate*matriz #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].pesos = pesos[:,i]
    
    def actualizar_bias(self, vector_b, rate):
        biases = np.array([n.bias for n in self.neuronas])
        biases -= rate*vector_b #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].bias = biases[i]
    
    #Funcion para poblar datos de entrada a la red.
    def set_capa(self, activaciones, vector_z):
        for i in range(len(self.neuronas)):
            self.neuronas[i].set_activacion(activaciones[i], vector_z[i])
            
    def mostrar_capa(self):
        for neurona in self.neuronas:
            print(neurona, end="\t")
        print("\n-----------------------------")

        
class Neurona:
    def __init__(self, tam_capa_anterior):
        self.pesos = np.random.rand(tam_capa_anterior) # genera una lista de numeros aleatorios de tamaño "tam_capa_anterior"
        self.bias = np.random.rand()
    def set_activacion(self, valor_a, valor_z):
        self.activacion = valor_a
        self.z = valor_z
        
    def __str__(self):
        return "Activación: "+str(self.activacion)

### ¡Es hora de probar!

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import keras
x_train,y_train = load_iris(return_X_y=True)
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train) 
y_onehot = keras.utils.to_categorical(y_train)
#transform target to one hot vector
nn = RedNeuronal(4, [32, 16], 3)
nn.train(x_train, y_onehot, 100, 0.5)


Using TensorFlow backend.


[ 2.895789   -0.80803189 -8.32259239]
[ -5.50349816e+04  -3.39004232e+01   3.38782498e+01]
[   -739.13385022     737.38180993 -145995.60551985]
[   236.27683067 -10862.71199974   -239.57111072]
[ -8.29901555e+04  -3.89681187e+01   7.99233421e+01]
[ -8.91547125e+02   8.93546199e+02  -3.91226737e+08]
[   6442.6132364    -6488.06284681 -919983.19718198]
[-3727.15598557   274.27924071  -141.37381801]
[ -2.33050633e+01  -6.84809261e+08   4.86529554e+01]
[  1.38348390e+07  -1.39909091e+07  -1.24020889e+09]
[  -1.17267333    1.10248843 -137.98429732]
[  -7.3661854  -440.37386649   16.54163079]
[  7.16502092e+00  -5.02807183e+00  -2.80668303e+08]
[  -8027.27608239    7927.5920644  -638543.98401158]
[  149.80911287 -2846.38094715   -75.92223402]
[ -7.89755959e+08  -3.04712267e+01   6.29752335e+01]
[ -1.46136909e+07   2.89052490e+07  -1.31129123e+09]
[  179.40255707 -8207.16964017   -89.68700478]
[ -9.16527597e+08  -7.51931745e+01   1.52399633e+02]
[ -9.29192783e+06   9.20309679e+06  -9.62643429

C:\Users\thena\Anaconda2\envs\rna\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in double_scalars



[   889.2453474    -897.43029786 -78695.54200802]
[   889.22807259   -897.41296231 -78692.96900196]
[  1779.65476062   -899.01139319 -78633.33141553]
[  1780.39165742   -899.38112269 -78688.3114619 ]
[  1780.38487459   -899.37773932 -78687.65381026]
[   889.25775309   -899.44386066 -78695.37689744]
[   889.19397228   -897.37877838 -78687.61283769]
[   889.23354373   -897.41845167 -78693.79161647]
[   889.25312195   -897.43809891 -78696.70641317]
[  1780.24925678   -899.3096718  -78677.70888196]
[   889.25487956   -899.44097534 -78694.96178306]
Error del epoch 1 : 1.95
[   889.2552869    -899.44138349 -78695.02718039]
[  1780.25221412   -899.31116089 -78677.8892072 ]
[  1779.60960835   -898.98886347 -78629.0095089 ]
[   889.25723034   -899.44333553 -78695.30311767]
[   889.25229451   -899.43837891 -78694.59431164]
[  1780.29787431   -899.33409192 -78681.13160771]
[   889.25777928   -899.44388694 -78695.38084787]
[   889.25251864   -899.43860541 -78694.61557736]
[   889.2225614    -897.

[   889.24493103   -897.42987581 -78695.51218183]
[   889.25251082   -899.43859545 -78694.63055482]
[   888.69433359   -896.87778669 -78610.1252348 ]
[  1773.78469706   -896.06922688 -78172.50104278]
[   889.00525755   -897.18958096 -78658.12583292]
[   889.18990052   -897.37465184 -78687.31538719]
[   889.25563396   -899.44173206 -78695.07671266]
[   889.25406787   -897.43904791 -78696.84917904]
[   889.25530935   -899.44140613 -78695.02969502]
[   889.24323435   -899.42928823 -78693.23458605]
[  1780.38487459   -899.37773932 -78687.65381026]
[  1779.74079596   -899.05472137 -78638.52125418]
[   889.0542656    -897.23866573 -78666.15434418]
[   889.25099028   -899.43706711 -78694.42277825]
[   889.24760583   -897.43256312 -78695.88686656]
[   889.21976911   -897.40462248 -78691.7868595 ]
[   889.24385451   -899.42990692 -78693.35487294]
[   889.23716814   -899.42321039 -78692.25711312]
[  1780.40550114   -899.38808084 -78689.25107405]
[   889.25487956   -899.44097534 -78694.96178306]


...

[   889.25301415   -899.4391019  -78694.69523409]
[  1779.74079596   -899.05472137 -78638.52125418]
[  1780.05878892   -899.21417148 -78663.00306175]
[  1780.40550114   -899.38808084 -78689.25107405]
[  1780.1833428    -899.27663875 -78672.4964142 ]
[   889.24501327   -897.42995935 -78695.51671398]
[   889.24426239   -899.43031656 -78693.41321277]
[  1780.17628778   -899.27301229 -78672.63202623]
[   889.25293803   -897.43791401 -78696.68155157]
[   889.22431054   -897.40919375 -78692.35736758]
[   889.25563432   -899.44173244 -78695.07660921]
[   889.24760583   -897.43256312 -78695.88686656]
[  1780.34607318   -899.3582343  -78685.04005362]
[   889.23354373   -897.41845167 -78693.79161647]
[   889.24788471   -899.43395263 -78693.94489386]
[   889.20953934   -897.39437171 -78690.14879357]
[   889.25392665   -899.44001858 -78694.82370816]
[   889.14460577   -897.32925511 -78680.13161943]
[  1779.09224879   -898.72932365 -78590.11724249]
[   889.25775309   -899.44386066 -78695.37689744]


[  1780.40837759   -899.38951742 -78689.51648125]
[   889.24396594   -897.4289089  -78695.35646099]
[   889.25668874   -899.4427914  -78695.22722403]
[   889.25530935   -899.44140613 -78695.02969502]
[  1779.67671432   -899.02265305 -78633.10016987]
[   888.15714987   -896.33952077 -78524.00722986]
[   889.24323435   -899.42928823 -78693.23458605]
[  1779.77756839   -899.07306196 -78642.10092431]
[  1780.25221412   -899.31116089 -78677.8892072 ]
[   889.25312195   -897.43809891 -78696.70641317]
[  1779.74079596   -899.05472137 -78638.52125418]
[  1780.39270492   -899.38167147 -78688.21234899]
[   889.25485347   -899.44094812 -78694.96584354]
[   889.00525755   -897.18958096 -78658.12583292]
[   889.25679829   -899.44290166 -78695.24115694]
[   889.23593689   -897.42086057 -78694.09215812]
[   889.16381705   -899.34964897 -78680.96915674]
[   889.21976911   -897.40462248 -78691.7868595 ]
[   889.25710043   -899.44320505 -78695.28473234]
[   889.25099028   -899.43706711 -78694.42277825]


[   889.18990052   -897.37465184 -78687.31538719]
[   889.25563432   -899.44173244 -78695.07660921]
[   889.25475002   -897.43973323 -78696.94491425]
[  1779.88790684   -899.12844927 -78650.13225767]
[  1780.45917126   -899.41501414 -78693.21832562]
[  1779.30603786   -898.83680008 -78604.45945666]
[   889.21976911   -897.40462248 -78691.7868595 ]
[   889.2432641    -897.42820334 -78695.2614659 ]
[   889.20953934   -897.39437171 -78690.14879357]
[  1780.38487459   -899.37773932 -78687.65381026]
[   889.25353204   -897.43851055 -78696.76664724]
[  1780.40837759   -899.38951742 -78689.51648125]
[   889.24426239   -899.43031656 -78693.41321277]
[   889.24385451   -899.42990692 -78693.35487294]
[   889.2552869    -899.44138349 -78695.02718039]
[   889.23716814   -899.42321039 -78692.25711312]
[   889.25611954   -899.44221937 -78695.14881709]
[   889.25401339   -899.44010466 -78694.84401448]
[  1780.27969672   -899.32495601 -78679.8937438 ]
[  1780.05878892   -899.21417148 -78663.00306175]


[   889.23752288   -897.42244265 -78694.40083767]
[  1779.6028304    -898.98563117 -78627.21804119]
[   889.25563396   -899.44173206 -78695.07671266]
[   889.25710043   -899.44320505 -78695.28473234]
[   889.25099028   -899.43706711 -78694.42277825]
[  1780.27969672   -899.32495601 -78679.8937438 ]
[   888.92803833   -897.11199014 -78647.39044361]
[  1780.39165742   -899.38112269 -78688.3114619 ]
[   889.25777928   -899.44388694 -78695.38084787]
[   889.25487956   -899.44097534 -78694.96178306]
[   889.20008968   -897.38489462 -78688.69705515]
[   889.24477594   -897.42972398 -78695.46006624]
[  1780.40550114   -899.38808084 -78689.25107405]
[  1780.24925678   -899.3096718  -78677.70888196]
[   888.69433359   -896.87778669 -78610.1252348 ]
[   889.25267984   -897.4376546  -78696.64551775]
[   889.24592928   -899.43199364 -78693.62833236]
[   889.25301415   -899.4391019  -78694.69523409]
[  1780.38487459   -899.37773932 -78687.65381026]
[  1780.1833428    -899.27663875 -78672.4964142 ]


KeyboardInterrupt: 

## 1. Referencias


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Pregunta 2

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer=SGD(lr=0.1),loss="mse", metrics=["accuracy"])

In [ ]:
model.fit(x_train, y_onehot, epochs=100, batch_size=1, verbose=1)